# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [30]:
from pathlib import Path
import warnings
import gc

import pandas as pd
import numpy as np


In [31]:
DATA_DIR = Path(__name__).resolve().parent.parent / "data"
print(f'DATA_DIR: {DATA_DIR}')

DATA_FILES = (
    "tracks.parquet",
    "catalog_names.parquet",
    "interactions.parquet"
)
print(f'DATA_FILES: {DATA_FILES}')

data_pathes = {name.split(".")[0]: DATA_DIR / name for name in DATA_FILES}
print(f'data_pathes: {data_pathes}')

warnings.filterwarnings("ignore")

RANDOM_STATE = 20241104
np.random.seed(RANDOM_STATE)

UNIQUE_USERS_SHARE = 0.25

pd.options.display.max_rows = 1024
pd.options.display.max_columns = 1024
pd.set_option("display.max_colwidth", 128)

DATA_DIR: /home/mle-user/mle_projects/mle-project-sprint-4-v001/data
DATA_FILES: ('tracks.parquet', 'catalog_names.parquet', 'interactions.parquet')
data_pathes: {'tracks': PosixPath('/home/mle-user/mle_projects/mle-project-sprint-4-v001/data/tracks.parquet'), 'catalog_names': PosixPath('/home/mle-user/mle_projects/mle-project-sprint-4-v001/data/catalog_names.parquet'), 'interactions': PosixPath('/home/mle-user/mle_projects/mle-project-sprint-4-v001/data/interactions.parquet')}


In [32]:
def view_df(df: pd.DataFrame) -> None:
    df.info()
    display(df.head())
    print(f'Shape: {df.shape}\n')
    
    try:
        print(f'Duplicates: {df.duplicated().sum()}\n')

    except Exception as e:
        print(f'Failed to count duplicates: {e}\n')

    print(f'Null-values:\n{df.isna().sum()}')


def load_view_parquet(path: Path, ) -> pd.DataFrame:
    df = pd.read_parquet(path)
    
    view_df(df)

    return df

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

## tracks.parquet

In [33]:
tracks = load_view_parquet(data_pathes["tracks"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 2490809, 6007655, 17294156]",[84],[11]


Shape: (1000000, 4)

Failed to count duplicates: unhashable type: 'numpy.ndarray'

Null-values:
track_id    0
albums      0
artists     0
genres      0
dtype: int64


In [34]:
null_tracks_ids = tracks[
    (np.vectorize(len)(tracks["albums"]) == 0)
    & (np.vectorize(len)(tracks["artists"]) == 0)
    & (np.vectorize(len)(tracks["genres"]) == 0)
]

display(null_tracks_ids.head(2))
null_tracks_ids.shape

,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]


(18, 4)

In [35]:
tracks = (
    tracks
    .drop(index=null_tracks_ids.index)
    .explode("albums")
    .explode("artists")
    .explode("genres")
    .astype(
        {
            "track_id": "int32",
            "albums": "Int32",
            "artists": "Int32",
            "genres": "Int16",
        }
    )
    .rename(
        columns={
            "albums": "album_id",
            "artists": "artist_id",
            "genres": "genre_id",
        }
    )
)

view_df(tracks)

<class 'pandas.core.frame.DataFrame'>
Index: 6765293 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Dtype
---  ------     -----
 0   track_id   int32
 1   album_id   Int32
 2   artist_id  Int32
 3   genre_id   Int16
dtypes: Int16(1), Int32(2), int32(1)
memory usage: 161.3 MB


,track_id,album_id,artist_id,genre_id
0,26,3,16,11
0,26,3,16,21
0,26,2490753,16,11
0,26,2490753,16,21
1,38,3,16,11


Shape: (6765293, 4)

Duplicates: 0

Null-values:
track_id         0
album_id         0
artist_id    41771
genre_id     36657
dtype: int64


## catalog_names.parquet

In [36]:
catalog_names = load_view_parquet(data_pathes["catalog_names"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack


Shape: (1812471, 3)

Duplicates: 0

Null-values:
id      0
type    0
name    0
dtype: int64


In [37]:
catalog_names = (
    catalog_names
    .sort_values(by="id")
    .reset_index(drop=True)
    .astype(
        {
            "id": "int32",
            "type": "category",
            "name": "string"
        }
    )
)

view_df(catalog_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype   
---  ------  -----   
 0   id      int32   
 1   type    category
 2   name    string  
dtypes: category(1), int32(1), string(1)
memory usage: 22.5 MB


,id,type,name
0,0,genre,all
1,1,genre,eastern
2,2,genre,rusrock
3,3,album,Taller Children
4,3,genre,rusrap


Shape: (1812471, 3)

Duplicates: 0

Null-values:
id      0
type    0
name    0
dtype: int64


## interactions.parquet

In [ ]:
all_interactions = load_view_parquet(data_pathes["interactions"])

users = all_interactions["user_id"].unique()
selected_users = np.random.choice(users, size=int(users.shape[0] * UNIQUE_USERS_SHARE), replace=False)

print(f'All users amount: {users.shape[0]}. Selected users amount: {selected_users.shape[0]}\n')

interactions = all_interactions[all_interactions["user_id"].isin(selected_users)]

del all_interactions, users, selected_users
_ = gc.collect()
print(f'Deleted all interactions data and unique users. The number of unreachable objects: {_}\n')

view_df(interactions)

All users amount: 1373221. Selected users amount: 343305
Deleted all interactions data and unique users.
<class 'pandas.core.frame.DataFrame'>
Index: 56077812 entries, 0 to 24
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 1.4 GB


,user_id,track_id,track_seq,started_at
0,3,6006252,1,2022-02-05
1,3,21642261,2,2022-02-10
2,3,21642265,3,2022-02-17
3,3,24692821,4,2022-03-02
4,3,25995209,5,2022-03-13


Shape: (56077812, 4)

Duplicates: 0

Null-values:
user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64


## Мерджинг

In [ ]:
data = tracks.copy()

for catalog in catalog_names["type"].unique():
    curr_id_ = f'{catalog}_id'
    curr_data_ = (
        catalog_names[catalog_names["type"] == catalog]
        .rename(columns={"id": curr_id_, "type": f'{catalog}_type', "name": f'{catalog}_name'})
    )

    data = data.merge(
        curr_data_,
        on=curr_id_,
        how="left"
    )


print("ok")

del curr_id_, curr_data_

data = data.merge(
    interactions,
    on="track_id",
    how="right"
)

view_df(data)

In [ ]:
1/0

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [ ]:
interactions.groupby("user_id")["track_id"].count().describe(percentiles=(p / 100 for p in range(0, 105, 5))).astype("string")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [ ]:
def plot_histogram_and_boxplot(data, column, category_column=None, hist_bins=100, width=800, height=800):
    title = f'Histogram and Boxplot of {column}'
    xlabel = column
    ylabel_hist='Amount'
    ylabel_box='Value'

    ratio = [0.382, 0.618] if category_column is not None else [0.25, 0.75]
    
    fig = sp.make_subplots(rows=2, cols=1, row_heights=ratio, shared_xaxes=True,
                           subplot_titles=(f'Boxplot of {column}', f'Histogram of {column}, {hist_bins} bins'))

    if category_column is None:
        fig.add_trace(
            go.Histogram(x=data[column], nbinsx=hist_bins, name='Histogram'),
            row=2,
            col=1
        )
        fig.add_trace(
            go.Box(x=data[column], name='Boxplot', width=0.5),
            row=1,
            col=1
        )
    else:
        unique_categories = data[category_column].unique()
        for category in unique_categories:
            category_data = data[data[category_column] == category]
            fig.add_trace(
                go.Histogram(x=category_data[column], nbinsx=hist_bins, name=f'Histogram - {category}', opacity=0.75),
                row=2,
                col=1
            )
            fig.add_trace(
                go.Box(x=category_data[column], name=f'Boxplot - {category}', width=0.5),
                row=1,
                col=1
            )

    fig.update_layout(
        height=height,
        width=width,
        title=title,
        showlegend=True,
        xaxis=dict(title=xlabel),
        xaxis2=dict(title=xlabel),
        yaxis=dict(title=ylabel_box),
        yaxis2=dict(title=ylabel_hist)
    )

    fig.show()

In [ ]:
plot_histogram_and_boxplot(
    pd.DataFrame(interactions.groupby("user_id")["track_id"].count()),
    "track_id"
)

In [ ]:
(
    interactions
    .groupby("started_at")
    [["user_id", "track_id", "track_seq"]]
    .count()
    .plot(title="Количество Прослушиваний")
);

# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.